In [41]:
import torch
from torch_geometric_temporal.dataset import METRLADatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split
import numpy as np


In [42]:
DEVICE = torch.device('cuda') # cuda
shuffle=True
batch_size = 64

In [43]:
# Loading dataset

loader = METRLADatasetLoader()

dataset = loader.get_dataset()

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio = 0.8)

In [44]:
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

In [45]:
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import DCRNN
from tqdm import tqdm

In [46]:
print("Cuda Available?: ", torch.cuda.is_available())
print("Current Device: ", torch.cuda.get_device_name(torch.cuda.current_device()))

Cuda Available?:  True
Current Device:  NVIDIA GeForce GTX 1060 with Max-Q Design


In [47]:
# for batches
#https://github.com/benedekrozemberczki/pytorch_geometric_temporal/blob/master/examples/recurrent/a3tgcn2_example.py
train_input = np.array(train_dataset.features) # (27399, 207, 2, 12)
train_target = np.array(train_dataset.targets) # (27399, 207, 12)
train_x_tensor = torch.from_numpy(train_input).type(torch.FloatTensor).to(DEVICE)  # (B, N, F, T)
train_target_tensor = torch.from_numpy(train_target).type(torch.FloatTensor).to(DEVICE)  # (B, N, T)
train_dataset_new = torch.utils.data.TensorDataset(train_x_tensor, train_target_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset_new, batch_size=batch_size, shuffle=shuffle,drop_last=True)


test_input = np.array(test_dataset.features) # (, 207, 2, 12)
test_target = np.array(test_dataset.targets) # (, 207, 12)
test_x_tensor = torch.from_numpy(test_input).type(torch.FloatTensor).to(DEVICE)  # (B, N, F, T)
test_target_tensor = torch.from_numpy(test_target).type(torch.FloatTensor).to(DEVICE)  # (B, N, T)
test_dataset_new = torch.utils.data.TensorDataset(test_x_tensor, test_target_tensor)
test_loader = torch.utils.data.DataLoader(test_dataset_new, batch_size=batch_size, shuffle=shuffle,drop_last=True)

In [51]:
#https://github.com/benedekrozemberczki/pytorch_geometric_temporal/blob/master/examples/recurrent/dcrnn_example.py
class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = DCRNN(node_features, 64, 5)
        self.linear = torch.nn.Linear(64, 12)

    # x needs to be 207 x 12
    def forward(self, x, edge_index, edge_weight):
        # x is B, N, F, T
        x = x.permute(0,1,3,2)
        # x is B, N, T, F
        #print('x_in shape: ', x_in.shape)
        # get just speed
        x = x[:,:,:,0]
        #reshape to (B*N, T)
        x = x.reshape((x.shape[0]*x.shape[1], x.shape[2]))
        h = self.recurrent(x, edge_index, edge_weight)
        h = F.relu(h)
        h = self.linear(h)
        return h

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = 'cpu'

model = RecurrentGCN(node_features = 12)

model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
lr_decay_ratio=0.1
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=steps, gamma=lr_decay_ratio)



loss_fn = torch.nn.MSELoss()
loss_fn2 = F.l1_loss

In [52]:
# Loading the graph once because it's a static graph
for snapshot in train_dataset:
    static_edge_index = snapshot.edge_index.to(DEVICE)
    static_edge_attr = snapshot.edge_attr.to(DEVICE)
    break

# Training the model 
model.train()

for epoch in range(100):
    step = 0
    loss_list = []
    for encoder_inputs, labels in train_loader:
        y_hat = model(encoder_inputs, static_edge_index, static_edge_attr)         # Get model predictions
        # reshape back to BxNxT
        y_hat = y_hat.reshape((labels.shape))
        mean = [53.59967, 0.4982691]
        std = [20.209862, 0.28815305]
        labels = labels*std[0] + mean[0]
        y_hat = y_hat*std[0] + mean[0]
        loss = loss_fn2(y_hat, labels) 
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        step= step+ 1
        #loss = torch.sqrt(loss)
        loss_list.append(loss.item())
        if step % 100 == 0 :
            print("    Loss: ", sum(loss_list)/len(loss_list))
    lr_scheduler.step()
    print("Epoch {} train MAE: {:.4f}".format(epoch, sum(loss_list)/len(loss_list)))


## Evaluation

#- Lets get some sample predictions for a specific horizon (e.g. 288/12 = 24 hours)
#- The model always gets one hour and needs to predict the next hour

model.eval()
step = 0
# Store for analysis
total_loss = []
for encoder_inputs, labels in test_loader:
    # Get model predictions
    y_hat = model(encoder_inputs, static_edge_index, static_edge_attr)
    y_hat = y_hat.reshape((labels.shape))
    # undo z-score
    mean = [53.59967, 0.4982691]
    std = [20.209862, 0.28815305]
    labels = labels*std[0] + mean[0]
    y_hat = y_hat*std[0] + mean[0]
    # Mean squared error
    #loss = loss_fn(y_hat, labels)
    # Mean absolute error
    loss = loss_fn2(y_hat, labels)
    total_loss.append(loss.item())
    # Store for analysis below
    #test_labels.append(labels)
    #predictions.append(y_hat)
    

print("Test MAE: {:.4f}".format(sum(total_loss)/len(total_loss)))



    Loss:  6.22374852180481
    Loss:  5.665551233291626
    Loss:  5.421403424739838
    Loss:  5.3200044733285905
Epoch 0 train MAE: 5.2954
    Loss:  4.846310782432556
    Loss:  4.850142118930816


KeyboardInterrupt: 

In [50]:
# no batching
# model.train()

# for epoch in tqdm(range(10)):
#     cost = 0
#     for time, snapshot in enumerate(train_dataset):
#         if(time == 0):
#             edge_idx = snapshot.edge_index.to(device)
#             edge_attr = snapshot.edge_attr.to(device)
#         x_in = snapshot.x.permute(0,2,1)
#         #print('x_in shape: ', x_in.shape)
#         x_in = x_in[:,:,0]
#         x_in = x_in.reshape((x_in.shape[0], x_in.shape[1]))
#         n,f,t = snapshot.x.shape  # n = num nodes, f = num features, t = num timesteps
        
#         #x_in = snapshot.x.reshape((t, n*f))
#         #print('new x_in shape: ', x_in.shape)
#         x_in = x_in.to(device)
        
#         y_hat = model(x_in, edge_idx, edge_attr)
#         y = snapshot.y.to(device)
#         cost = cost + torch.mean((y_hat-y)**2)
#         #print("y_hat shape: ", y_hat.shape)
#         #print("y shape: ", y.shape)
#         #print(time)
#     cost = cost / (time+1)
#     cost.backward()
#     optimizer.step()
#     optimizer.zero_grad()
    
# model.eval()
# cost = 0
# for time, snapshot in enumerate(test_dataset):
#     x_in = snapshot.x.permute(0,2,1)
#     #print('x_in shape: ', x_in.shape)
#     x_in = x_in[:,:,0]
#     x_in = x_in.reshape((x_in.shape[0], x_in.shape[1]))
#     x_in = x_in.to(device)
#     edge_idx = snapshot.edge_index.to(device)
#     edge_attr = snapshot.edge_attr.to(device)
#     y_hat = model(x_in, edge_idx, edge_attr)
#     y = snapshot.y.to(device)
#     #print("y_hat shape: ", y_hat.shape)
#     #print("y shape: ", y.shape)
#     cost = cost + torch.mean((y_hat-y)**2)
# cost = cost / (time+1)
# cost = cost.item()
# print("MSE: {:.4f}".format(cost))